In [1]:
import csv
import string
import unicodedata
import pandas as pd
import json

In [2]:
data = pd.read_csv('./data/topics_geolocated.csv', encoding='latin-1')

In [3]:
data.head()

,id_prose,lat,lng,RegiÃ³n,comuna,relato,fecha,marca,tipo,topic_0,topic_1,topic_2,topic_3
0,54954,-706.891.713,-33.425.017.499.999.900,RegiÃ³n Metropolitana de Santiago,Quinta Normal,deje EstaciÃ³nado camioneta mas menos media ho...,08/29/2013,CHEVROLET,CAMIONETA,0.011905,0.011907,0.964281,0.011906
1,54956,-705.755.745,-334.065.217,RegiÃ³n Metropolitana de Santiago,Las Condes,deje vehiculo EstaciÃ³nado aproximadamente 0...,05-09-13,SUZUKI,AUTOMOVIL,0.000000,0.000000,0.984535,0.000000
2,54960,-702.144.494,-332.368.381,RegiÃ³n Metropolitana de Santiago,San JosÃ© de MaipÃº,detenido abrir puerta bajarme acerca persona a...,09/20/2013,KIA MOTORS,AUTOMOVIL,0.000000,0.000000,0.000000,0.983693
3,54961,-7.075.141.352.790.460,-3.348.327.125,RegiÃ³n Metropolitana de Santiago,MaipÃº,mientras encontraba vehiculo EstaciÃ³nado af...,09/21/2013,CHEVROLET,AUTOMOVIL,0.440833,0.000000,0.554335,0.000000
4,54963,-7.060.492.190.000.000,-33.417.865.899.999.900,RegiÃ³n Metropolitana de Santiago,Providencia,arrendamos vehiculo yculo dya 17092013 1830 l...,09/17/2013,SAMSUNG,AUTOMOVIL,0.000000,0.000000,0.898415,0.094285


In [4]:
mean = data.groupby('comuna').mean().reset_index()

In [5]:
mean.head()

,comuna,id_prose,topic_0,topic_1,topic_2,topic_3
0,AlhuÃ©,47802.500000,0.000000,0.000000,0.983694,0.000000
1,Buin,62528.361702,0.110589,0.064583,0.467013,0.349556
2,Calera de Tango,68201.595745,0.176521,0.166498,0.445316,0.204666
3,Cerrillos,59629.783505,0.110429,0.123642,0.540764,0.216621
4,Cerro Navia,60595.362069,0.076887,0.072169,0.545298,0.296922


In [6]:
std = data.groupby('comuna').std()

In [18]:
comunas = {}
for index, row in mean.iterrows():
    comunas[row['comuna']] = {}
    for topic in ['topic_0', 'topic_1', 'topic_2', 'topic_3']:
        comunas[row['comuna']][topic] = round(row[topic], 2)

In [23]:
with open('comunas_dist.json', 'w') as fp:
    json.dump(comunas, fp)

In [9]:
data['formatted_fecha'] = pd.to_datetime(data['fecha'])

In [11]:
data['formatted_fecha']

0       2013-08-29
1       2013-05-09
2       2013-09-20
3       2013-09-21
4       2013-09-17
5       2013-09-16
6       2013-09-21
7       2013-10-10
8       2013-10-13
9       2013-10-13
10      2013-07-10
11      2013-10-19
12      2013-10-22
13      2013-10-19
14      2013-10-30
15      2013-10-29
16      2013-07-11
17      2013-11-13
18      2013-11-13
19      2013-05-11
20      2013-11-18
21      2013-11-15
22      2013-11-26
23      2013-11-28
24      2013-11-28
25      2013-12-27
26      2013-12-31
27      2014-07-01
28      2014-11-01
29      2014-12-01
           ...    
17599   2015-03-20
17600   2015-11-03
17601   2016-11-01
17602   2016-07-24
17603   2011-02-13
17604   2011-04-14
17605   2011-06-16
17606   2012-04-27
17607   2012-02-08
17608   2013-08-19
17609   2013-05-09
17610   2013-12-28
17611   2015-08-10
17612   2015-06-11
17613   2015-10-12
17614   2016-01-30
17615   2016-06-04
17616   2016-06-04
17617   2016-04-20
17618   2011-07-31
17619   2014-07-20
17620   2015

In [36]:
mapped_data = data.groupby([data.formatted_fecha.dt.year, data.comuna]).mean().reset_index()

In [38]:
mapped_data.head()

,formatted_fecha,comuna,id_prose,topic_0,topic_1,topic_2,topic_3
0,2006,Quinta Normal,37171.0,0.000000,0.732023,0.263471,0.000000
1,2008,Conchali,37170.0,0.051384,0.076449,0.000000,0.867701
2,2010,Colina,31157.0,0.000000,0.000000,0.989795,0.000000
3,2010,Independencia,33506.0,0.000000,0.000000,0.919579,0.072546
4,2010,La Florida,38518.0,0.000000,0.000000,0.981707,0.000000


In [43]:
years = [2011, 2012, 2013, 2014, 2015, 2016]
dframes = []
for year in years:
    dframes.append(mapped_data[mapped_data['formatted_fecha'] == year])

In [50]:
for i in range(0, len(years)):
    comunas = {}
    for index, row in dframes[i].iterrows():
        comunas[row['comuna']] = {}
        for topic in ['topic_0', 'topic_1', 'topic_2', 'topic_3']:
            comunas[row['comuna']][topic] = round(row[topic], 2)
    with open('comunas_dist_{}.json'.format(years[i]), 'w') as fp:
        json.dump(comunas, fp)

In [6]:
content = []
with open('words.txt') as f:
    content = f.readlines()
    content = [x.strip() for x in content]

In [7]:
content

['(\'0.039*"robo" + 0.033*"total" + 0.026*"danos" + 0.019*"circunstancia" + 0.019*"senalizacion" + 0.019*"presencia" + 0.018*"4" + 0.017*"alcoholemiano" + 0.016*"carabinerosno" + 0.011*"hechos" + 0.010*"vehiculo" + 0.009*"chapa" + 0.008*"externo" + 0.008*"puerta" + 0.007*"daÿÿos" + 0.006*"web" + 0.006*"call" + 0.006*"denunciado" + 0.006*"delantero" + 0.006*"trasero"\')',
 '(\'0.054*"auto" + 0.041*"vehiculo" + 0.019*"si" + 0.015*"algún" + 0.013*"llaves" + 0.013*"camioneta" + 0.012*"estacionamiento" + 0.011*"robado" + 0.011*"cuenta" + 0.009*"todas" + 0.008*"seguridad" + 0.008*"comprobante" + 0.007*"carabineros" + 0.007*"mecanismo" + 0.006*"camion" + 0.006*"asegurado" + 0.006*"dÿa" + 0.006*"empresa" + 0.005*"denuncia" + 0.004*"patente"\')',
 '(\'0.099*"vehiculo" + 0.050*"estacionado" + 0.022*"deje" + 0.016*"robado" + 0.013*"robo" + 0.013*"lugar" + 0.012*"ÿculo" + 0.011*"auto" + 0.011*"encontraba" + 0.011*"calle" + 0.010*"hrs" + 0.010*"domicilio" + 0.010*"volver" + 0.009*"carabineros" + 0.

In [20]:
topic_index = 0
words_json = []
for topic in content:
    document = {}
    document["topic"] = topic_index
    document["words"] = []
    words = topic.split("+")
    for word in words:
        words_hash = {}
        items = word.split("*")
        words_hash["word"] = items[1].replace("(", " ").replace(")", " ").replace('"', " ").replace("'", " ").strip()
        words_hash["prob"] = items[0].replace("(", " ").replace(")", " ").replace('"', " ").replace("'", " ").strip()
        document["words"].append(words_hash)
    topic_index += 1
    words_json.append(document)

In [21]:
words_json

[{'topic': 0,
  'words': [{'word': 'robo', 'prob': '0.039'},
   {'word': 'total', 'prob': '0.033'},
   {'word': 'danos', 'prob': '0.026'},
   {'word': 'circunstancia', 'prob': '0.019'},
   {'word': 'senalizacion', 'prob': '0.019'},
   {'word': 'presencia', 'prob': '0.019'},
   {'word': '4', 'prob': '0.018'},
   {'word': 'alcoholemiano', 'prob': '0.017'},
   {'word': 'carabinerosno', 'prob': '0.016'},
   {'word': 'hechos', 'prob': '0.011'},
   {'word': 'vehiculo', 'prob': '0.010'},
   {'word': 'chapa', 'prob': '0.009'},
   {'word': 'externo', 'prob': '0.008'},
   {'word': 'puerta', 'prob': '0.008'},
   {'word': 'daÿÿos', 'prob': '0.007'},
   {'word': 'web', 'prob': '0.006'},
   {'word': 'call', 'prob': '0.006'},
   {'word': 'denunciado', 'prob': '0.006'},
   {'word': 'delantero', 'prob': '0.006'},
   {'word': 'trasero', 'prob': '0.006'}]},
 {'topic': 1,
  'words': [{'word': 'auto', 'prob': '0.054'},
   {'word': 'vehiculo', 'prob': '0.041'},
   {'word': 'si', 'prob': '0.019'},
   {'word'